<a href="https://colab.research.google.com/github/cosmo3769/SSL-study/blob/classifier2/classifier_iNaturalist_aves.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and Imports

In [5]:
# Check if the notebook is running on colab or not.
try:
    import google.colab
    COLAB = True
except:
    COLAB = False
print(f'Is the notebook using colab environment: {COLAB}')

if COLAB:
    # Install W&B for MLOPs.
    print(f'Installing Weights and Biases')
    !pip install --upgrade wandb

# Login to your W&B account.
import wandb
wandb.login()

Is the notebook using colab environment: False


True

In [15]:
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models

# Prepare Dataset

In [72]:
def download_dataset(dataset_name: str, dataset_type: str, version: str='latest'):
    """
    Utility function to download the data saved as W&B artifacts and return a dataframe
    with path to the dataset and associated label.
    
    Args:
        dataset_name (str): The name of the dataset - `train`, `val`, `out-class`, and `in-class`.
        dataset_type (str): The type of the dataset - `labelled-dataset`, `unlabelled-dataset`.
        version (str): The version of the dataset to be downloaded. By default it's `latest`,
            but you can provide different version as `vX`, where, X can be 0,1,...
            
        Note that the following combination of dataset_name and dataset_type are valid:
            - `train`, `labelled-dataset`
            - `val`, `labelled-dataset`
            - `in-class`, `unlabelled-dataset`
            - `out-class`, `unlabelled-dataset`
            
    Return:
        df_data (pandas.DataFrame): Dataframe with path to images with associated labels if present.
    """
    # Download the dataset.
    wandb_api = wandb.Api()
    artifact = wandb_api.artifact(f'ayush-thakur/ssl-study-data/{dataset_name}:{version}', type=dataset_type)
    artifact_dir = artifact.download()
    
    # Open the W&B table downloaded as a json file.
    json_file = glob.glob(artifact_dir+'/*.json')
    assert len(json_file) == 1
    with open(json_file[0]) as f:
        data = json.loads(f.read())
        assert data['_type'] == 'table'
        columns = data['columns']
        data = data['data']

    # Create a dataframe with path and label
    df_columns = ['image_path', 'width', 'height']
    if 'label' in columns:
        df_columns+=['label']
    data_df = pd.DataFrame(columns=df_columns)

    for idx, example in tqdm(enumerate(data)):
        image_dict = example[1]
        image_path = os.path.join(train_dir, image_dict.get('path'))
        height = image_dict.get('height')
        width = image_dict.get('width')
        
        df_data = [image_path, width, height]
        if 'label' in columns:
            df_data+=[example[2]]
        data_df.loc[idx] = df_data
    
    # Shuffle the dataframe
    data_df = data_df.sample(frac=1).reset_index(drop=True)

    return data_df

In [73]:
%%time
train_df = download_dataset('train', 'labelled-dataset')
valid_df = download_dataset('val', 'labelled-dataset')

wandb: Downloading large artifact train:latest, 351.17MB. 3960 files... Done. 0:0:0
3959it [00:07, 523.00it/s]
wandb: Downloading large artifact val:latest, 184.02MB. 2001 files... Done. 0:0:0
2000it [00:03, 524.56it/s]

CPU times: user 12.9 s, sys: 462 ms, total: 13.3 s
Wall time: 12.8 s


In [77]:
train_df.head()

,image_path,width,height,label
0,./artifacts/train:v0/media/images/e18792a1abc3...,500,446,116
1,./artifacts/train:v0/media/images/c7029a52a870...,500,375,12
2,./artifacts/train:v0/media/images/9d7f38451468...,500,357,127
3,./artifacts/train:v0/media/images/590770a32881...,500,425,96
4,./artifacts/train:v0/media/images/c9114d607dc3...,500,375,16


## Build an input pipeline with tf.data

In [24]:
# Import necessary libraries

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import applications
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, GlobalMaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

### Training input pipeline

In [25]:
training_labelled['file_name'] = training_labelled['file_name'].str.replace(r'trainval_images/', '')
training_labelled['file_name'] = '/content/train/train/' + training_labelled['file_name'].str[:]
training_dataframe = training_labelled.drop(['image_id', 'id', 'width', 'height'], axis = 1)
training_dataframe

,category_id,file_name
0,0,/content/train/train/0/0.jpg
1,0,/content/train/train/0/1.jpg
2,0,/content/train/train/0/2.jpg
3,0,/content/train/train/0/3.jpg
4,0,/content/train/train/0/4.jpg
...,...,...
3954,199,/content/train/train/199/1.jpg
3955,199,/content/train/train/199/2.jpg
3956,199,/content/train/train/199/3.jpg
3957,199,/content/train/train/199/4.jpg


In [26]:
# Coverting DataFrame to list to pass it to tf.data dataloader

file_name = training_dataframe['file_name'].to_list()
label = training_dataframe['category_id'].to_list()

In [27]:
train_slice = tf.data.Dataset.from_tensor_slices((file_name, label))

In [28]:
for feature_batch, label_batch in train_slice.take(1):
  print("'category': {}".format(label_batch))
  print("'feature': {}".format(feature_batch))

'category': 0
'feature': b'/content/train/train/0/0.jpg'


In [29]:
# Decode the images and resize the images to the mentioned shape

@tf.function
def parse_function(filename, label):
    image_string = tf.io.read_file(filename)

    #Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.io.decode_jpeg(image_string, channels=3)

    #This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)

    image = tf.image.resize(image, [224, 224])
    return image, label

In [ ]:
# def train_preprocess(image, label):
#     image = tf.image.random_flip_left_right(image)

#     image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
#     image = tf.image.random_saturation(image, lower=0.5, upper=1.5)

#     #Make sure the image is still in [0, 1]
#     image = tf.clip_by_value(image, 0.0, 1.0)

#     return image, label

# # Apply training augmentation
# train_transforms = Compose([
#             Resize(224, 224, p=1),
#             Rotate(limit=20),
#             Cutout(num_holes=8, max_h_size=30, max_w_size=30, p=1.0),
#             HorizontalFlip(p=0.7),
#             VerticalFlip(p=0.4),
#             Normalize(
#                 mean=[0.485, 0.456, 0.406],
#                 std=[0.229, 0.224, 0.225],
#                 max_pixel_value=255.0,
#                 p=1.0,
#             ),
#         ])

# # Apply validation augmentation
# valid_transforms = Compose([
#             Resize(224, 224, p=1),
#             Normalize(
#                 mean=[0.485, 0.456, 0.406],
#                 std=[0.229, 0.224, 0.225],
#                 max_pixel_value=255.0,
#                 p=1.0,
#             ),
#         ])

# def aug_train_fn(image):
#     data = {"image":image}
#     aug_data = train_transforms(**data)
#     aug_img = aug_data["image"]

#     return aug_img.astype(np.float32) 

# def aug_valid_fn(image):
#     data = {"image":image}
#     aug_data = valid_transforms(**data)
#     aug_img = aug_data["image"]

#     return aug_img.astype(np.float32) 

# def train_augmentations(image, label):
    
#     aug_img = tf.numpy_function(func=aug_train_fn, inp=[image], Tout=tf.float32)
#     aug_img.set_shape((224, 224, 3))

#     return aug_img, label

# def valid_augmentations(image, label):

#     aug_img = tf.numpy_function(func=aug_valid_fn, inp=[image], Tout=tf.float32)
#     aug_img.set_shape((224, 224, 3))

#     return aug_img, label

In [30]:
AUTOTUNE = tf.data.AUTOTUNE

training_dataset = train_slice.shuffle(len(file_name))
training_dataset = training_dataset.map(parse_function, num_parallel_calls=AUTOTUNE)
# training_dataset = training_dataset.map(train_preprocess, num_parallel_calls=AUTOTUNE)
training_dataset = training_dataset.batch(32)
training_dataset = training_dataset.prefetch(AUTOTUNE)

In [31]:
inputs, labels = next(iter(training_dataset))
labels

<tf.Tensor: shape=(32,), dtype=int32, numpy=
array([110,  59,  96,  81,  91,  45, 167,  79, 140,  58, 102,  63,  54,
        13,  32,  83,  73,  86,  74,  26,  50,  54,  16, 111, 159, 191,
        14,   2,  37,  67,   5,  13], dtype=int32)>

### Validation input pipeline

In [32]:
validation_labelled['file_name'] = validation_labelled['file_name'].str.replace(r'trainval_images/', '')
validation_labelled['file_name'] = '/content/val/val/' + validation_labelled['file_name'].str[:]
validation_dataframe = validation_labelled.drop(['image_id', 'id', 'width', 'height'], axis = 1)
validation_dataframe

,category_id,file_name
0,0,/content/val/val/0/30.jpg
1,0,/content/val/val/0/31.jpg
2,0,/content/val/val/0/32.jpg
3,0,/content/val/val/0/33.jpg
4,0,/content/val/val/0/34.jpg
...,...,...
1995,199,/content/val/val/199/11.jpg
1996,199,/content/val/val/199/12.jpg
1997,199,/content/val/val/199/13.jpg
1998,199,/content/val/val/199/14.jpg


In [33]:
# Coverting DataFrame to list to pass it to tf.data dataloader

val_file_name = validation_dataframe['file_name'].to_list()
val_label = validation_dataframe['category_id'].to_list()

In [34]:
val_slices = tf.data.Dataset.from_tensor_slices((val_file_name, val_label))

In [35]:
for feature_batch, label_batch in val_slices.take(1):
  print("'category': {}".format(label_batch))
  print("'feature': {}".format(feature_batch))

'category': 0
'feature': b'/content/val/val/0/30.jpg'


In [36]:
AUTOTUNE = tf.data.AUTOTUNE

# validation_dataset = val_slices.shuffle(len(val_file_name))
validation_dataset = val_slices.map(parse_function, num_parallel_calls=AUTOTUNE)
# validation_dataset = validation_dataset.map(train_preprocess, num_parallel_calls=AUTOTUNE)
validation_dataset = validation_dataset.batch(32)
validation_dataset = validation_dataset.prefetch(AUTOTUNE)

In [37]:
inputs, labels = next(iter(training_dataset))
labels

<tf.Tensor: shape=(32,), dtype=int32, numpy=
array([143,  43, 134,  97,   4,  15,  34,  31,  85, 124,  58,   7,  33,
        59, 181,   5,  11,  86, 134,  80,  65, 136, 145, 106,  23,  56,
       187,  92, 147,  26,  28, 163], dtype=int32)>

## Build Model Architecture

In [41]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
  
base_model.trainable = False

model = tf.keras.Sequential([
                             base_model,
                             tf.keras.layers.GlobalMaxPooling2D(),
                             tf.keras.layers.Dense(200, activation='softmax')
])

model.summary() 

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_max_pooling2d_1 (Glo  (None, 512)              0         
 balMaxPooling2D)                                                
                                                                 
 dense_1 (Dense)             (None, 200)               102600    
                                                                 
Total params: 14,817,288
Trainable params: 102,600
Non-trainable params: 14,714,688
_________________________________________________________________


## Compile and Train the Model

In [42]:
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
        training_dataset,
        epochs = 20,
        validation_data = validation_dataset,
        callbacks=[
          # Stopping our training if val_accuracy doesn't improve after 20 epochs
          tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=15),
    ]
)

## Visualization of model's prerformance

In [ ]:
import matplotlib.pyplot as plt

# store results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
  
  
# plot results
# accuracy
plt.figure(figsize=(10, 16))
plt.rcParams['figure.figsize'] = [16, 9]
plt.rcParams['font.size'] = 14
plt.rcParams['axes.grid'] = True
plt.rcParams['figure.facecolor'] = 'white'
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')

# loss
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.xlabel('epoch')
plt.tight_layout(pad=3.0)
plt.show()

## Evaluation on Validation dataset 

In [ ]:
accuracy_score = model.evaluate(validation_dataset)
print(accuracy_score)
print("Accuracy: {:.4f}%".format(accuracy_score[1] * 100)) 
  
print("Loss: ",accuracy_score[0])

## Prediction on Test dataset

In [ ]:
images_annotations_test['file_name'] = '/content/test/' + images_annotations_test['file_name'].str[:]
testing_dataframe = images_annotations_test.drop(['width', 'height', 'id'], axis = 1)
testing_dataframe

In [ ]:
file_name = testing_dataframe['file_name'].to_list()

In [ ]:
test_slice = tf.data.Dataset.from_tensor_slices(file_name)

In [ ]:
for feature_batch in test_slice.take(1):
  print("'feature': {}".format(feature_batch))

In [ ]:
# Decode the images and resize the images to the mentioned shape

@tf.function
def test_parse_function(filename):
    image_string = tf.io.read_file(filename)

    #Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.io.decode_jpeg(image_string, channels=3)

    #This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)

    image = tf.image.resize(image, [224, 224])
    return image

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

test_dataset = test_slice.map(test_parse_function, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(32)
test_dataset = test_dataset.prefetch(AUTOTUNE)

In [ ]:
inputs = next(iter(training_dataset))
inputs